# olefile - http://code-angie.tistory.com/49

In [ ]:
# 아래한글 문서를 바로 txt 문서로 변환하여 읽어 올 수 있는 방법
import olefile
# "E:\ai\Bidding-Mate\data\raw\file\(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp"
path = r"E:\ai\Bidding-Mate\data\processed\test.hwp"
f = olefile.OleFileIO(path)#아래한글 문서 열기
# 아래한글 문서 디코딩
print(f.listdir())


[['\x05HwpSummaryInformation'], ['BinData', 'BIN0001.png'], ['BinData', 'BIN0002.jpg'], ['BinData', 'BIN0003.bmp'], ['BinData', 'BIN0004.bmp'], ['BinData', 'BIN0005.jpg'], ['BinData', 'BIN0006.jpg'], ['BinData', 'BIN0007.jpg'], ['BinData', 'BIN0008.jpg'], ['BinData', 'BIN0009.jpg'], ['BodyText', 'Section0'], ['DocInfo'], ['DocOptions', '_LinkDoc'], ['FileHeader'], ['PrvImage'], ['PrvText'], ['Scripts', 'DefaultJScript'], ['Scripts', 'JScriptVersion']]


In [2]:
# openstream 함수로 문서의 내용을 꺼내고, 유니코드로 인코딩된 문서를 utf-16으로 디코딩을 하는 것이다.
# 한글 파일 안에는 다양한 디렉토리가 존재하는데, 이중 PrvText 와 BodyText를 이용하는 방법

# PrvText로 문서 읽기
# PrvText는 예상할 수 있듯이 미리보기를 사용하여 문서를 읽는 방법이다.
# 만약 문서 길이가 1페이지 내외라면 간단하게 내용을 읽을 수 있다.
encoded_txt = f.openstream("PrvText").read()
text = encoded_txt.decode("utf-16",errors="ignore")
print(text)



<><>
<2024년 ｢벤처확인종합관리시스템 기능 고도화｣ 용역사업 (복수의결권주식, 스톡옵션, 성과조건부주식) - 제안요청서>
<><>

2024. 03.


<><<><><><><목  차><><><><>><>
<><>
< 1. 추진개요  3      2. 추진방안  5      3. 추진내용  9      4. 제안요청내용  24      5. 입찰관련사항  78      6. 제안서작성요령  82   7. 별지서식 및 붙임  94  >
<Ⅰ. 추진개요>

<1><><추진배경 및 방향>

□ 「벤처기업육성에 관한 특별조치법」(이하 ‘벤처기업법’) 복수의결권주식, 스톡옵션(주식매수선택권), 성과조건부주식교부계약(RS) 등의 기능 고도화 및 이관, 신규 구축업무를 本 과업에서 추진

  ⚬  (복수의결권주식*) 벤처기업법 제16조의11에 따라 발행된 복수의결권주식 보고 업무처리 시스템 구축

      * 복수의결권이란? 모든 주주는 1주당 하나의 의결권을 갖는 주주평등원칙(상법)과 별도로 비상장 벤처기업 창업자에게만 1주당 최대 10배의 의결권 행사를 부여하는 제도 

  ⚬  (스톡옵션*) 벤처기업법 제16조의3에 따라 부여된 벤처기업 스톡옵션 부여, 취소·철회 신고 및 업무시스템 구축

     * 스톡옵션이란? 비상장 벤처기업 임·직원 및 기업 성장에 기여한 자에게 미리 정한 가격(행사가격)으로 신주를 인수하거나 자기의 주식을 매수할 수 있는 권리 혹은 주식의 시가와 행사가격의 차액을 청구할 수 있는 권리를 부여하는 제도

     -  중소벤처24 內 스톡옵션 신청 기능 이관을 추진하며, 과거 데이터 정제 및 적재 등을 통한 데이터 이관 추진

  ⚬  (성과조건부주식*) 벤처기업법 제16조의19에 따라 성과조건부주식교부계약의 신고 및 업무처리 시스템 구축

     * 성과조건부주식이란? 비상장 벤처기업 임ㆍ직원 및 특정인에게 자기주식 부여를 약속하되, 일정 재직기간 및 조건을 충족해야 주식이 부여 대상자에게 귀속되도록 제한을 둔 


In [3]:
import olefile
import zlib
import struct
#### 추가 ####
import re
import unicodedata

class HWPExtractor(object):
    FILE_HEADER_SECTION = "FileHeader"
    HWP_SUMMARY_SECTION = "\x05HwpSummaryInformation"
    SECTION_NAME_LENGTH = len("Section")
    BODYTEXT_SECTION = "BodyText"
    HWP_TEXT_TAGS = [67]

    def __init__(self, filename):
        self._ole = self.load(filename)
        self._dirs = self._ole.listdir()

        self._valid = self.is_valid(self._dirs)
        if (self._valid == False):
            raise Exception("Not Valid HwpFile")
        
        self._compressed = self.is_compressed(self._ole)
        self.text = self._get_text()
	
    # 파일 불러오기 
    def load(self, filename):
        return olefile.OleFileIO(filename)
	
    # hwp 파일인지 확인 header가 없으면 hwp가 아닌 것으로 판단하여 진행 안함
    def is_valid(self, dirs):
        if [self.FILE_HEADER_SECTION] not in dirs:
            return False

        return [self.HWP_SUMMARY_SECTION] in dirs

	# 문서 포맷 압축 여부를 확인
    def is_compressed(self, ole):
        header = self._ole.openstream("FileHeader")
        header_data = header.read()
        return (header_data[36] & 1) == 1

	# bodytext의 section들 목록을 저장
    def get_body_sections(self, dirs):
        m = []
        for d in dirs:
            if d[0] == self.BODYTEXT_SECTION:
                m.append(int(d[1][self.SECTION_NAME_LENGTH:]))

        return ["BodyText/Section"+str(x) for x in sorted(m)]
	
    # text를 뽑아내는 함수
    def get_text(self):
        return self.text

	# 전체 text 추출
    def _get_text(self):
        sections = self.get_body_sections(self._dirs)
        text = ""
        for section in sections:
            text += self.get_text_from_section(section)
            text += "\n"

        self.text = text
        return self.text

	# section 내 text 추출
    def get_text_from_section(self, section):
        bodytext = self._ole.openstream(section)
        data = bodytext.read()

        unpacked_data = zlib.decompress(data, -15) if self.is_compressed else data
        size = len(unpacked_data)

        i = 0

        text = ""
        while i < size:
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            level = (header >> 10) & 0x3ff
            rec_len = (header >> 20) & 0xfff

            if rec_type in self.HWP_TEXT_TAGS:
                rec_data = unpacked_data[i+4:i+4+rec_len]
                
                ############## 정제 추가된 부분 #############
                decode_text = rec_data.decode('utf-16')
                # 문자열을 담기 전 정제하기
                res = remove_control_characters(remove_chinese_characters(decode_text))
                
                text += res
                text += "\n"

            i += 4 + rec_len

        return text

#################### 텍스트 정제 함수 #######################
# 중국어 제거
def remove_chinese_characters(s: str):   
    return re.sub(r'[\u4e00-\u9fff]+', '', s)
    
# 바이트 문자열 제거
def remove_control_characters(s):    
    return "".join(ch for ch in s if unicodedata.category(ch)[0]!="C")
    
        
# text 추출 함수 -> 이 함수를 사용하면 됨
def get_text(filename):
    hwp = HWPExtractor(filename) 
    return hwp.get_text()

In [4]:
get_text(path)

'\n\n2024년 ｢벤처확인종합관리시스템 기능 고도화｣ 용역사업\n(복수의결권주식, 스톡옵션, 성과조건부주식) -\n제안요청서\n2024. 03.\n\n\n\n\n목  차\n 1. 추진개요ȃ 3\n   \n 2. 추진방안ȃ 5\n   \n 3. 추진내용ȃ 9\n   \n 4. 제안요청내용ȃ 24\n   \n 5. 입찰관련사항ȃ 78\n   \n 6. 제안서작성요령ȃ 82\n 7. 별지서식 및 붙임ȃ 94\n\nⅠ. 추진개요\n\n1\n추진배경 및 방향\n□ 「벤처기업육성에 관한 특별조치법」(이하 ‘벤처기업법’) 복수의결권주식, 스톡옵션(주식매수선택권), 성과조건부주식교부계약(RS) 등의 기능 고도화 및 이관, 신규 구축업무를  과업에서 추진\n  ⚬(복수의결권주식*) 벤처기업법 제16조의11에 따라 발행된 복수의결권주식 보고 업무처리 시스템 구축\n      * 복수의결권이란? 모든 주주는 1주당 하나의 의결권을 갖는 주주평등원칙(상법)과 별도로 비상장 벤처기업 창업자에게만 1주당 최대 10배의 의결권 행사를 부여하는 제도 \n  ⚬(스톡옵션*) 벤처기업법 제16조의3에 따라 부여된 벤처기업 스톡옵션 부여, 취소·철회 신고 및 업무시스템 구축\n     * 스톡옵션이란? 비상장 벤처기업 임·직원 및 기업 성장에 기여한 자에게 미리 정한 가격(행사가격)으로 신주를 인수하거나 자기의 주식을 매수할 수 있는 권리 혹은 주식의 시가와 행사가격의 차액을 청구할 수 있는 권리를 부여하는 제도\n -중소벤처24  스톡옵션 신청 기능 이관을 추진하며, 과거 데이터 정제 및 적재 등을 통한 데이터 이관 추진\n  ⚬(성과조건부주식*) 벤처기업법 제16조의19에 따라 성과조건부주식교부계약의 신고 및 업무처리 시스템 구축\n     * 성과조건부주식이란? 비상장 벤처기업 임ㆍ직원 및 특정인에게 자기주식 부여를 약속하되, 일정 재직기간 및 조건을 충족해야 주식이 부여 대상자에게 귀속되도록 제한을 둔 주식\n□ 복수의결권주식 발행 보고 및 스톡옵션 신고기능은 기존 시스템 내 벤처확

한자, 이상한 알파벳 또는 문자들 섞여 있음.

- 목 차 \n\n1. 일반과업지시서ȃ3\n1.1 과 업 명 : 용인 첨단 시스템반도체 국가산단 용수공급사업 타당성조사 및 기본계획 수립 용역Ԋȃ3\n1.2 과업의 목적ȃ3\n1.3 과업대상지역ȃ3\n1.4 사업개요ȃ3\n1.5 과업수행기간ȃ3\n1.6 과업의 범위 및 내용䜘ȃ
- nྠĀྠĀྠĀྠĀྠĀྠĀ
- ĀྠĀྠĀྠĀ  
- n╣ॣ╣ॣ